In [1]:
import sys, os
if 'google.colab' in sys.modules:
    %cd
    % rm -rf MPyDATA
    ! git clone --recurse-submodules -j8 https://github.com/Michaeldz36/MPyDATA.git
    %cd MPyDATA
    ! git checkout develop
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from MPyDATA_examples.Olesik_et_al_2020.setup import Setup
from MPyDATA_examples.Olesik_et_al_2020.demo_plot_convergence  import plot
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3, x_log_of_pn
from MPyDATA.options import Options
from MPyDATA_examples.Olesik_et_al_2020.simulation import Simulation
from MPyDATA_examples.utils.show_plot import show_plot
from joblib import Parallel, parallel_backend, delayed
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA.arakawa_c.discretisation import discretised_analytical_solution
from MPyDATA_examples.utils.error_norms import L2, Smolarkiewicz_Grabowski_1990_eq21

In [3]:
GCs = np.linspace(.05,.95, 7)
nrs = np.linspace(64,256, 7, dtype = int)

In [4]:
grid_layout = x_p2()
psi_coord = x_p2()
setup = Setup()

In [5]:
def analysis(nr, GC, opt):
    setup = Setup(nr = nr, mixing_ratios_g_kg = [1.2,])
    options = Options(**opt)
    simulation = Simulation(setup, grid_layout,  psi_coord, options, GC)
    simulation.step(simulation.out_steps[-1])
    t = simulation.out_steps[-1] * simulation.dt
    rh = simulation.rh
    pdf_t = PdfEvolver(setup.pdf, setup.drdt, t)
    analytical = discretised_analytical_solution(
                simulation.rh.magnitude,
                lambda r: pdf_t(r * rh.units).magnitude
            ) * pdf_t(rh[0]).units
    numerical = simulation.n
    error = np.log(Smolarkiewicz_Grabowski_1990_eq21(numerical.magnitude, analytical.magnitude, t.magnitude)) / np.log(2)
    return nr, GC, error

In [6]:
opt_set = (
    {'n_iters': 1},
     {'n_iters': 2},
     {'n_iters': 2,'infinite_gauge':True},
     {'n_iters': 2, 'infinite_gauge': True, 'flux_corrected_transport': True},
     {'n_iters': 2, 'third_order_terms':True},
     {'n_iters': 3},
     {'n_iters': 3, 'third_order_terms': True, 'infinite_gauge': True, 'flux_corrected_transport': True}
)


In [7]:
for opt in opt_set:
    with parallel_backend('threading', n_jobs=-2):
        results0 = Parallel(verbose=10)(
            delayed(analysis)(nr, GC, opt)
            for nr in nrs
            for GC in GCs
        )
    results = [list(i) for i in zip(*results0)]
    plot_setup = np.array(results[0:2])
    measures = {'log$_2$(err)':results[2]}
    for measure in measures.keys():
        plot(plot_setup[0], plot_setup[1], measures[measure], name = measure)
        show_plot(filename = f'convergence_{measure}.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  5.9min remaining:  1.1min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  6.6min remaining:   25.7s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  7.0min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  7.3min remaining:  1.4min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  8.3min remaining:   32.3s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  8.8min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   47.6s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  7.9min remaining:  1.5min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  8.8min remaining:   34.5s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  9.4min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed: 12.9min remaining:  2.5min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed: 14.5min remaining:   56.6s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed: 15.4min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:   45.7s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  7.6min remaining:  1.5min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed:  8.6min remaining:   33.5s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed:  9.1min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed:  9.0min remaining:  1.8min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed: 10.1min remaining:   39.5s
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed: 10.8min finished
No handles with labels found to put in legend.


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed: 13.5min
[Parallel(n_jobs=-2)]: Done  41 out of  49 | elapsed: 15.5min remaining:  3.0min
[Parallel(n_jobs=-2)]: Done  46 out of  49 | elapsed: 17.8min remaining:  1.2min
[Parallel(n_jobs=-2)]: Done  49 out of  49 | elapsed: 19.0min finished
No handles with labels found to put in legend.
